<h1><center>emopok xgboost 🌳 </center></h1>
<center>authors: [Aina Nurmagombetova](https://github.com/anurma) 🤙 [Alina Cherepanova](https://github.com/alinacherepanova) 🙋 [Anya Bataeva](https://github.com/fyzbt) 🤯 [Olya Silyutina](https://github.com/olgasilyutina) 🤩</center>

In [79]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier

In [2]:
# prepare data
unique_df = pd.read_csv('./data/unique_emopok.csv')
textfeatures_df = pd.read_csv('./data/textfeatures_emopok.csv')
sent_df = pd.read_csv('./data/sentiments_emopok.csv')
topics_df = pd.read_csv('./data/dum_topics_emopok.csv')
emoji_df = pd.read_csv('./data/emoji_texts_df.csv')
emo_clusters = pd.read_csv('./data/emopok_clusters.csv')
d2v_vectors_df = pd.read_csv('./data/d2v_vectors_emopok.csv')

In [3]:
sent_df.columns = ['index', 'sent', 'texts']
xgb_df = unique_df.merge(textfeatures_df.drop(['text'], axis=1), on = 'index')
xgb_df = xgb_df.merge(sent_df.drop('texts', axis = 1), on = 'index')
xgb_df = pd.concat([xgb_df.reset_index(drop=True), d2v_vectors_df], axis=1)
xgb_df = pd.concat([xgb_df.reset_index(drop=True), topics_df], axis=1)
emoji_df = emoji_df[['emoji', 'index']].drop_duplicates()
xgb_df = xgb_df.merge(emoji_df, on = 'index')
emo_clusters.columns = ['emoji', 'cluster_group']
xgb_df = xgb_df.merge(emo_clusters, on = 'emoji')
xgb_df = xgb_df.drop(['emoji', 'texts'], axis = 1)

In [4]:
len(xgb_df)

547642

In [5]:
xgb_df.head(1)

,index,n_chars,n_commas,n_digits,n_exclaims,n_hashtags,n_lowers,n_mentions,n_urls,n_words,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,cluster_group
0,1,755,3,123,0,1,301,0,3,666,...,0,0,0,0,0,0,0,0,0,14


In [6]:
xgb_df.to_csv('./data/xgb_emopok.csv', index = False)

In [2]:
xgb_df = pd.read_csv('./data/xgb_emopok.csv')

In [5]:
# cols = [c for c in xgb_df.columns if 'topic' in c.lower()]
# xgb_df = xgb_df.drop(cols, axis = 1)

In [80]:
for i in tqdm(['16_3', '6', '3', '1', '23']):
    xgb_df_sample = xgb_df[xgb_df['cluster_group'] == i].sample(40000)
    xgb_df = xgb_df[xgb_df['cluster_group'] != i]
    xgb_df = pd.concat([xgb_df, xgb_df_sample])

In [81]:
xgb_df = xgb_df.reset_index(drop=True)

In [84]:
# top clusters
xgb_df.groupby('cluster_group').index.count().sort_values('index', ascending = False).reset_index().head(10)

,cluster_group,index
0,16_3,40000
1,1,40000
2,6,40000
3,3,40000
4,23,40000
5,21,29775
6,2,28627
7,13,16723
8,16_7,16192
9,18,15311


In [244]:
Y = xgb_df[['cluster_group']]
X = xgb_df.drop(['cluster_group', 'index'], axis = 1)

In [245]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [246]:
model = XGBClassifier(n_jobs=4, silent=False, objective='multi:softprob', n_estimators=1)

In [89]:
eval_set = [(X_test, y_test.values.ravel())]
model.fit(X_train, y_train.values.ravel(), eval_set=eval_set, verbose=True)

[0]	validation_0-merror:0.866463


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1,
       n_jobs=4, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [208]:
sorted_idx = np.argsort(model.feature_importances_)[::-1]
cols = []
imp = []
for index in sorted_idx:
    cols.append(X_train.columns[index])
    imp.append(model.feature_importances_[index])

In [222]:
feature_importances = pd.DataFrame({'features': cols, 'importances': imp})

In [223]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [224]:
feature_importances = feature_importances[feature_importances['importances'] > 0.01]
feature_importances = feature_importances.sort_values('importances')

In [252]:
trace = go.Bar(
    y = list(feature_importances['features']),
    x = list(feature_importances['importances']),
    marker=dict(color='#3AA2FB'),
    orientation = 'h')

layout = dict(title = '',
              width=400,
              height=400,
              xaxis = dict(title = ''),
              yaxis = dict(title = ''))

fig = dict(data=[trace])
iplot(fig)

In [92]:
preds = model.predict(X_test)

/Users/o.silutina/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, preds)

In [ ]:
X_test['predictions'] = preds
test_indexes = xgb_df.iloc[X_test.index]
test_indexes = test_indexes[['index']].reset_index()
test_predictions = X_test.reset_index()[['index', 'predictions']]
test_predictions.columns = ['level_0', 'predictions']
test_indexes = test_indexes.merge(test_predictions, on = 'level_0')
fin_test = unique_df.merge(test_indexes, on = 'index').sort_values('level_0')\
    .merge(emoji_df, on = 'index', how = 'inner')\
    .merge(emo_clusters, on = 'emoji', how = 'inner')

In [116]:
fin_test = fin_test.drop(['emoji', 'level_0'], axis = 1).drop_duplicates()

In [ ]:
fin_test = fin_test.assign(NE=fin_test.predictions.astype(str) == fin_test.cluster_group)

In [164]:
fin_test.to_csv('./data/success_pred.csv', index = False)

In [178]:
good_ones = fin_test[(fin_test['NE'] == True)]

In [ ]:
# fin_test[(fin_test['NE'] == True) & (~fin_test['predictions'].isin(['16_3', '14', '3', '1', '12', '16_1', '15_7',\
#                                                                    '15_0', '2', '21', '13', '6', '23', '24', '5',\
#                                                                    '20']))]

In [196]:
for i in [249744, 70842, 323158, 323158, 296003, 182053, 104306, 216050, 347738, 39268, 208593, 249754, 153240,\
         337200, 73613, 136165, 272755, 10123, 327009, 155539, 297794, 281368, 150979, 43400, 210014, 20756, 158854,\
         153096, 332217]:
    prediction_ind = str(fin_test[fin_test['index'] == i]['predictions'].tolist()[0])
    prediction = ' '.join(set(emo_clusters[(emo_clusters['cluster_group'] == prediction_ind) & \
                                      (~emo_clusters['emoji'].isin(['🐀', '🐸', '⚜']))]\
                          .sample(5, replace=True)['emoji'].tolist()))
    print('Сообщение: ' + str(fin_test[fin_test['index'] == i]['texts'].tolist()[0]) + \
          'Предсказание: ' + prediction
           + '\n')
    

Сообщение: Ну наконец-то эти пары закончились, как же я отвык за несколько месяцев 😓Предсказание: 👎 🔞 😓 🎆 👏

Сообщение: @lovvely_1d люди столько не живут😓Предсказание: 🖕 😤 👊 🔞 👏

Сообщение: Что делать если у тебя есть краш, но ты очень сильно боишься признаться ему🙁😓😓😓Предсказание: 🇷🇺 😥 ♥ 🔞 😓

Сообщение: Что делать если у тебя есть краш, но ты очень сильно боишься признаться ему🙁😓😓😓Предсказание: 💔 👊 🌊 😫

Сообщение: Судьба уберегла 😫😫😫🤐🤐🤐Предсказание: 😐 😟 💤 🤥 🙁

Сообщение: Да не ниче🤐Предсказание: 🤪 🧐 💤 🤥 🙁

Сообщение: @SirPhilipp Да блин не поймите неправильно😨Я вот человек взрослый мне 15(через 2 года)а вот моя мать это не понимает!!Я как истинная бабочка🦋покидаю свой кокон,-800 гр за месяц на голоде(параметры 120/80)и мама говорит что у меня анорексия,надо сделать вид при ней что кушаю чтобы отстала🙏Предсказание: 🎶 🌹 🎼 🎧 💐

Сообщение: Конец рабочего дня 💫💫💫💜💜💜 Поскорее бы закончился этот день 🙌🙌🙌 Жду завтрашний день  ммммм бабочки 🦋 в животе #curiousaboutARMY https://t.co/M4wCyyqEOlП